In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import time
from sklearn import *

/Users/georgewang/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/georgewang/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/georgewang/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.1

In [2]:
# a helper function to generate a file
def to_kaggle(prediction):
    current_time_str = time.strftime("%H-%M-%S_%a_%b_%d", time.localtime())
    file_name = "results/{}.txt".format(current_time_str)
    np.savetxt(file_name,
           np.vstack( (np.arange(len(prediction)) , prediction[:,1]) ).T,
           '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');


In [3]:
# load data
x_data_all = np.genfromtxt("data/X_train.txt", delimiter=None)
y_data_all = np.genfromtxt("data/Y_train.txt", delimiter=None)
x_test = np.genfromtxt("data/X_test.txt", delimiter=None)

In [4]:
# x_data = x_data_all[:10000, :]
# y_data = y_data_all[:10000]

x_data = x_data_all
y_data = y_data_all

In [5]:
# split training / validation data
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(
    x_data, y_data, test_size=0.2, random_state=42)

In [6]:
# logistic_classifier = pipeline.Pipeline([
#     ('poly', preprocessing.PolynomialFeatures(degree=2)),
#     ('linear', linear_model.LogisticRegression())])

# print("training started")
# logistic_classifier.fit(x_train, y_train)
# print("training finished")

# logistic_classifier_roc = metrics.roc_auc_score(y_validation, logistic_classifier.predict_proba(x_validation)[:,1])
# print(logistic_classifier_roc)

training started
training finished


In [9]:
# svm_classifier = pipeline.Pipeline([
#     ('feature_selection', feature_selection.SelectKBest(feature_selection.f_classif, k=4).fit(x_train, y_train)),
#     ('svm', svm.SVC(degree = 2, probability=True))])

# print("training started")
# svm_classifier.fit(x_train[:10000, :], y_train[:10000])
# print("training finished")

# svm_classifier_roc = metrics.roc_auc_score(y_validation, svm_classifier.predict_proba(x_validation)[:,1])
# print(svm_classifier_roc)

training started
training finished


In [ ]:
random_forest_classifier = ensemble.RandomForestClassifier(n_estimators=200, min_samples_leaf = 4)

print("training started")
starting_time = time.time()
random_forest_classifier.fit(x_train, y_train)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

random_forest_classifier_roc = metrics.roc_auc_score(
    y_validation, random_forest_classifier.predict_proba(x_validation)[:,1])
print(random_forest_classifier_roc)

training started
